In [ ]:
"""
Instructions to install packages and libraries:
1. !nvidia-smi (Check CUDA)
https://www.carc.usc.edu/user-guides/data-science/building-conda-environment
2. module purge
3. module load conda
4. mamba init bash
5. source ~/.bashrc
6. mamba create --name <env_name>
7. mamba activate <env_name>
8. mamba install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
9. mamba install other libraries
10. Run a batch job
"""

In [2]:
# Batch Job Script. May have to change the env
# #!/bin/bash

# #SBATCH --account=yzhao010_1531
# #SBATCH --partition=gpu
# #SBATCH --nodes=1
# #SBATCH --ntasks=1
# #SBATCH --cpus-per-task=4
# #SBATCH --gpus-per-task=a100:1
# #SBATCH --mem=16G
# #SBATCH --time=2:30:00
# #SBATCH --output=/home1/kharwal/logs_stdout.txt
# #SBATCH --error=/home1/kharwal/logs_stderr.txt

# module purge
# module load conda
# eval "$(conda shell.bash hook)"

# mamba activate /home1/kharwal/.conda/envs/dl_100

# python dl_100.py



In [ ]:
"""
Political Neutrality Model: Fine-tuning MPT-7B for neutral article generation
-------------------
This script fine-tunes a 7B parameter model to generate politically neutral
content given left-leaning and right-leaning articles using QLoRA techniques.
"""

import os
import json
import pandas as pd
import numpy as np
from itertools import cycle
from dataclasses import dataclass
from typing import Dict, List, Any
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

import evaluate
from tqdm import tqdm
import nltk
nltk.download('punkt_tab', quiet=True)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:

CONFIG = {
    "model_name": "mosaicml/mpt-7b-8k-instruct", 
    "root_dir": "/home1/kharwal/",              
    "output_dir": "mpt-lora-neutral",  

    "max_seq_length": 6000,                     
    "per_device_batch_size": 1,                 
    "gradient_accumulation_steps": 8,           
    "learning_rate": 2e-4,                      
    "num_train_epochs": 5,     

    "lora_r": 8,                                
    "lora_alpha": 16,                           
    "lora_dropout": 0.05,                       
    "target_modules": ["Wqkv", "out_proj"],     

    "weight_decay": 0.01,                       
    "warmup_steps": 25,                         

    "max_new_tokens": 1800,                     
    "evaluate_every": 30,                      
    "eval_accumulation_steps": 8,               
    "save_total_limit": 2,                      
    "save_strategy": "steps",                   

    "json_files": [                             
        "flipside_4K_articles_23-25.json",
        "flipside_4k_articles_21-22.json",
        "flipside_4k_articles_19-20.json",
    ]
}

In [ ]:
def load_all_data(file_paths: List[str], root_dir: str) -> List[Dict[str, Any]]:
    all_entries = []
    for path in file_paths:
        full_path = os.path.join(root_dir, path)
        if os.path.exists(full_path):
            print(f"Loading data from {full_path}")
            with open(full_path, "r") as f:
                all_entries.extend(json.load(f))
        else:
            print(f"[WARN] File not found: {full_path}")

    print(f"Loaded {len(all_entries)} total entries from {len(file_paths)} files")
    return all_entries

def generate_triplets(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    triplets = []

    for entry in data:
        left_articles = entry.get("left", [])
        right_articles = entry.get("right", [])
        neutral_articles = entry.get("neutral", [])

        if not (left_articles and right_articles and neutral_articles):
            continue

        left_cycle = cycle(left_articles)
        right_cycle = cycle(right_articles)

        for neutral in neutral_articles:
            left = next(left_cycle)
            right = next(right_cycle)

            left_combined = f"{left['headline']}. {left['text']}"
            right_combined = f"{right['headline']}. {right['text']}"
            neutral_combined = f"{neutral['headline']}. {neutral['text']}"

            triplets.append({
                "left": left_combined,
                "right": right_combined,
                "neutral": neutral_combined,
                "meta": {
                    "event_id": entry["id"],
                    "date": entry["date"],
                    "source_title": entry["source_title"]
                }
            })

    print(f"Generated {len(triplets)} triplets")
    return triplets

class TripletDataset(Dataset):

    def __init__(self, df, tokenizer, max_length=4096):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        prompt = f"<|left|>\n{row['left']}\n<|right|>\n{row['right']}\n<|neutral|>\n{row['neutral']}"

        encoding = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone()
        }

@dataclass
class DataCollator:

    tokenizer: AutoTokenizer

    def __call__(self, batch):
        input_ids = [b["input_ids"] for b in batch]
        attention_mask = [b["attention_mask"] for b in batch]
        labels = [b["labels"] for b in batch]

        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        attention_mask = torch.nn.utils.rnn.pad_sequence(
            attention_mask, batch_first=True, padding_value=0
        )
        labels = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=-100 
        )

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }


In [ ]:
def generate_neutral_article(left, right, model, tokenizer, max_length=4096, max_new_tokens=2048):
    prompt = f"<|left|>\n{left}\n<|right|>\n{right}\n<|neutral|>\n"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_length
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if "<|neutral|>" in generated_text:
        generated_neutral = generated_text.split("<|neutral|>")[-1].strip()
    else:
        generated_neutral = generated_text.split(prompt)[-1].strip()

    return generated_neutral

def evaluate_model(model, tokenizer, test_df, metrics_dict):
    model.eval()

    generated_list = []
    reference_list = []

    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
        gen = generate_neutral_article(
            row["left"],
            row["right"],
            model,
            tokenizer,
            max_length=CONFIG["max_seq_length"] - CONFIG["max_new_tokens"],
            max_new_tokens=CONFIG["max_new_tokens"]
        )
        ref = row["neutral"]

        generated_list.append(gen)
        reference_list.append(ref)

        if idx % 25 == 0:
            print(f"\nSample {idx}:")
            print(f"Generated: {gen}")
            print(f"Reference: {ref}")
            torch.cuda.empty_cache()

    rouge_result = metrics_dict["rouge"].compute(
        predictions=generated_list,
        references=reference_list,
        use_stemmer=True
    )

    tokenized_gen = [nltk.word_tokenize(text.lower()) for text in generated_list]
    tokenized_ref = [[nltk.word_tokenize(text.lower())] for text in reference_list]

    bleu_result = metrics_dict["bleu"].compute(
        predictions=tokenized_gen,
        references=tokenized_ref
    )

    print("\n===== EVALUATION RESULTS =====")
    print("ROUGE scores:", {k: v.mid.fmeasure for k, v in rouge_result.items()})
    print("BLEU score:", bleu_result["bleu"])

    results = {
        "rouge1": rouge_result["rouge1"].mid.fmeasure,
        "rouge2": rouge_result["rouge2"].mid.fmeasure,
        "rougeL": rouge_result["rougeL"].mid.fmeasure,
        "bleu": bleu_result["bleu"],
    }

    return results, generated_list, reference_list

In [ ]:
def main():
    output_dir = os.path.join(CONFIG["root_dir"], CONFIG["output_dir"])
    os.makedirs(output_dir, exist_ok=True)

    print("\n===== LOADING DATA =====")
    combined_data = load_all_data(CONFIG["json_files"], CONFIG["root_dir"])
    triplet_data = generate_triplets(combined_data)
    df_triplets = pd.DataFrame(triplet_data)

    print(f"Dataset size: {len(df_triplets)} examples")
    text_lengths = df_triplets.apply(
        lambda x: len(x['left'].split()) + len(x['right'].split()) + len(x['neutral'].split()),
        axis=1
    )
    print(f"Average total word count per example: {text_lengths.mean():.1f}")
    print(f"Max total word count per example: {text_lengths.max()}")


    output_text_lengths = df_triplets.apply(
        lambda x: len(x['neutral'].split()),
        axis=1
    )
    estimated_output_token_lengths = output_text_lengths * 1.3
    print(f"Estimated average token count per triplet: {estimated_output_token_lengths.mean():.1f}")
    print(f"Estimated max token count per triplet: {estimated_output_token_lengths.max():.1f}")
    print(f"Number of examples that may exceed 6500 tokens: {(estimated_output_token_lengths > 2048).sum()}")


    estimated_token_lengths = text_lengths * 1.3
    print(f"Estimated average token count per triplet: {estimated_token_lengths.mean():.1f}")
    print(f"Estimated max token count per triplet: {estimated_token_lengths.max():.1f}")
    print(f"Number of examples that may exceed 6500 tokens: {(estimated_token_lengths > 6500).sum()}")

    token_bins = [0, 2000, 4000, 6000, 8000, float('inf')]
    token_counts = pd.cut(estimated_token_lengths, bins=token_bins).value_counts().sort_index()
    print("Estimated token length distribution:")
    for i, count in enumerate(token_counts):
        if i < len(token_bins) - 1:
            print(f"  {token_bins[i]}-{token_bins[i+1]}: {count} examples")


    print("\n===== SPLITTING DATA =====")
    df_train_val, df_test = train_test_split(df_triplets, test_size=0.1, random_state=42)
    df_train, df_val = train_test_split(df_train_val, test_size=0.1, random_state=42)
    print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

    print("\n===== SETTING UP TOKENIZER =====")
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"], padding_side="right")
    tokenizer.pad_token = tokenizer.eos_token

    special_tokens_dict = {
        "additional_special_tokens": ["<|neutral|>", "<|left|>", "<|right|>"],
        "bos_token": "<|neutral|>"
    }
    tokenizer.add_special_tokens(special_tokens_dict)

    print("\n===== SETTING UP MODEL WITH QLORA =====")

    model = AutoModelForCausalLM.from_pretrained(
        CONFIG["model_name"],
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model.resize_token_embeddings(len(tokenizer))

    # for name, _ in model.named_modules():
    #     print(name)

    lora_config = LoraConfig(
        r=CONFIG["lora_r"],
        lora_alpha=CONFIG["lora_alpha"],
        target_modules=CONFIG["target_modules"],
        lora_dropout=CONFIG["lora_dropout"],
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    model.gradient_checkpointing_enable()
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    train_dataset = TripletDataset(df_train, tokenizer, max_length=CONFIG["max_seq_length"])
    val_dataset = TripletDataset(df_val, tokenizer, max_length=CONFIG["max_seq_length"])

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=CONFIG["per_device_batch_size"],
        per_device_eval_batch_size=CONFIG["per_device_batch_size"],
        gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
        eval_strategy="steps",
        eval_steps=CONFIG["evaluate_every"],
        save_strategy=CONFIG["save_strategy"],
        save_steps=CONFIG["evaluate_every"],
        logging_steps=10,
        learning_rate=CONFIG["learning_rate"],
        num_train_epochs=CONFIG["num_train_epochs"],
        warmup_steps=CONFIG["warmup_steps"],
        fp16=True,
        bf16=False,
        weight_decay=CONFIG["weight_decay"],
        save_total_limit=CONFIG["save_total_limit"],
        logging_dir=os.path.join(output_dir, "logs"),
        report_to="none",
        gradient_checkpointing=True,
        eval_accumulation_steps=CONFIG["eval_accumulation_steps"]
    )

    print("\n===== SETTING UP TRAINER =====")
    data_collator = DataCollator(tokenizer)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

    print("\n===== STARTING TRAINING =====")
    try:
        trainable = any(p.requires_grad for p in model.parameters())
        trainer.train()
        model.save_pretrained(os.path.join(output_dir, "adapters"))
        tokenizer.save_pretrained(os.path.join(output_dir, "tokenizer"))
        print(f"Model saved to {os.path.join(output_dir, 'adapters')}")
    except Exception as e:
        print(f"Training error: {e}")
        import traceback
        traceback.print_exc()

    print("\n===== EVALUATING MODEL =====")
    metrics = {
        "rouge": evaluate.load("rouge"),
        "bleu": evaluate.load("bleu")
    }

    model.eval()
    eval_results, generated_samples, reference_samples = evaluate_model(
        model, tokenizer, df_test, metrics
    )


    print("\n===== SAVING RESULTS =====")
    with open(os.path.join(output_dir, "eval_metrics.json"), "w") as f:
        json.dump(eval_results, f, indent=2)

    examples = []
    for i in range(len(generated_samples)):
        examples.append({
            "left": df_test.iloc[i]["left"],
            "right": df_test.iloc[i]["right"],
            "generated": generated_samples[i],
            "reference": reference_samples[i]
        })

    with open(os.path.join(output_dir, "sample_outputs.json"), "w") as f:
        json.dump(examples, f, indent=2)

    print(f"Evaluation results saved to {output_dir}")
    print("\n===== TRAINING COMPLETE =====")

if __name__ == "__main__":
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    main()